# FaceNet Regression

In [20]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_hub as hub
import pathlib
import PIL

## Retrieve All Data

In [6]:
img_height = 224
img_width = 224

In [7]:
train_txt = "data/regression_train_test/train_ratings.txt"
test_txt = "data/regression_train_test/test_ratings.txt"

with open(train_txt) as train_ratings:
    train_ratings = train_ratings.readlines()[0]
    train_ratings = np.array(train_ratings.split())
    train_ratings = train_ratings.astype(float)

with open(test_txt) as test_ratings:
    test_ratings = test_ratings.readlines()[0]
    test_ratings = np.array(test_ratings.split())
    test_ratings = test_ratings.astype(float)

print(train_ratings)
print(test_ratings)

[3.433333 2.9      3.966667 ... 3.05     2.4      2.666667]
[2.333333 3.45     2.7      ... 2.533333 4.433333 3.05    ]


In [8]:
def image_to_tensor(impath):
    img = PIL.Image.open(impath).resize((img_width, img_height))
    img = np.array(img)
    tensor = tf.image.convert_image_dtype(img, dtype=tf.float16, saturate=False, name=None)
    return tensor

In [9]:
train_dir = pathlib.Path("data/all_images_cropped/")
test_dir = pathlib.Path("data/all_images_cropped/")

train_images = list(train_dir.glob("*_train/*.jpg"))
train_images = sorted(train_images, key=lambda x: str(x)[str(x).index("_train")+7:])
test_images = list(test_dir.glob("*_test/*.jpg"))
test_images = sorted(test_images, key=lambda x: str(x)[str(x).index("_test")+6:])

x_train = np.array([image_to_tensor(impath) for impath in train_images])
y_train = train_ratings

x_test = np.array([image_to_tensor(impath) for impath in test_images])
y_test = test_ratings

In [10]:
def extract_ratings(text_path):
    with open(text_path) as ratings:
        ratings = ratings.readlines()[0]
        ratings = np.array(ratings.split())
        ratings = ratings.astype(float)

    return ratings

In [11]:
m_train_txt = "data/all_images_cropped/m_train_ratings"
f_train_txt = "data/all_images_cropped/f_train_ratings"

m_train_ratings = extract_ratings(m_train_txt)
f_train_ratings = extract_ratings(f_train_txt)

In [12]:
m_test_txt = "data/all_images_cropped/m_test_ratings"
f_test_txt = "data/all_images_cropped/f_test_ratings"

m_test_ratings = extract_ratings(m_test_txt)
f_test_ratings = extract_ratings(f_test_txt)

In [13]:
m_train_dir = pathlib.Path("data/all_images_cropped/m_train")
f_train_dir = pathlib.Path("data/all_images_cropped/f_train")
m_test_dir = pathlib.Path("data/all_images_cropped/m_test")
f_test_dir = pathlib.Path("data/all_images_cropped/f_test")

m_train_images = list(m_train_dir.glob("*.jpg"))
f_train_images = list(f_train_dir.glob("*.jpg"))
m_test_images = list(m_test_dir.glob("*.jpg"))
f_test_images = list(f_test_dir.glob("*.jpg"))

m_x_train = np.array([image_to_tensor(impath) for impath in m_train_images])
m_y_train = m_train_ratings

f_x_train = np.array([image_to_tensor(impath) for impath in f_train_images])
f_y_train = f_train_ratings

m_x_test = np.array([image_to_tensor(impath) for impath in m_test_images])
m_y_test = m_test_ratings

f_x_test = np.array([image_to_tensor(impath) for impath in f_test_images])
f_y_test = f_test_ratings

## Regression Without Gender Stratification

In [21]:
tfhub_url = "https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4"  # Example URL, replace with FaceNet URL
face_recognition_model = hub.load(tfhub_url)

In [22]:
def preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(160, 160))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    return img_array / 255.0  # Normalize pixel values to [0, 1]

In [23]:
def extract_embeddings(image_path):
    preprocessed_image = preprocess_image(image_path)
    embeddings = face_recognition_model(preprocessed_image)
    return embeddings.numpy()

In [27]:
image_path = "data/my_images/shakira.jpg"
embeddings = extract_embeddings(image_path)
print("Facial embeddings:", embeddings[0])

Facial embeddings: 2048
